In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.options.mode.chained_assignment = None

In [2]:
sample_submission = pd.read_csv("data/sample_submission.csv")
sample_submission.set_index("ID", inplace=True)

train = pd.read_csv("train_cleaned.csv")
test_X = train.loc[(train["date_block_num"] == train["date_block_num"].max())]
train_X = train.loc[(train["date_block_num"] < train["date_block_num"].max())]

pred_data = pd.read_csv("test_cleaned.csv")

train_y = train_X.pop("item_cnt_day")
test_y = test_X.pop("item_cnt_day")

train_X.drop(columns=["date_block_num"], inplace=True)
test_X.drop(columns=["date_block_num"], inplace=True)

In [3]:
model = LinearRegression()
model.fit(train_X.values, train_y.values)

LinearRegression()

In [4]:
round_test = False

test_predictions = model.predict(test_X)
if round_test:
    test_predictions = test_predictions.round()
err = np.absolute(test_predictions - test_y.values)
std_err = err.sum() / err.shape[0]
rmse = mean_squared_error(test_y.values, test_predictions, squared=False)
print(f"Model accuracy: {std_err}, RMSE: {rmse}")

Model accuracy: 1.3814830235900262, RMSE: 5.095005229373506


In [5]:
round_predictions = False

predictions = model.predict(pred_data.values)
if round_predictions:
    predictions = predictions.round()
sample_submission["item_cnt_month"] = predictions
sample_submission.to_csv("submission.csv")

In [6]:
train

,date_block_num,shop_id,item_id,item_cnt_day,item_price,month_of_year,item_category_id,prev_month_3,prev_month_2,prev_month_1
0,3,2,31,1.0,579.0,4,37,1.0,4.0,0.0
1,3,2,464,1.0,298.0,4,49,0.0,0.0,0.0
2,3,2,482,1.0,3230.0,4,73,1.0,1.0,1.0
3,3,2,484,3.0,293.0,4,73,2.0,4.0,2.0
4,3,2,485,2.0,297.0,4,73,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1421187,33,59,22087,6.0,88.0,10,83,3.0,2.0,5.0
1421188,33,59,22088,2.0,92.0,10,83,1.0,7.0,7.0
1421189,33,59,22091,1.0,128.0,10,83,3.0,0.0,0.0
1421190,33,59,22100,1.0,627.0,10,42,1.0,0.0,0.0
